## The first part of an LCEL chain must be a runnable type.

In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

bad_first_input = {
    "film_required_age": 18,
}

prompt = ChatPromptTemplate.from_template(
    "Generate a film title, the age is {film_required_age}"
)

# This will error:
bad_chain = bad_first_input | prompt

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'int'>

In [ ]:
# All of these chains enforce the runnable interface:
first_good_input = {"film_required_age": itemgetter("film_required_age")}

# Creating a dictionary within a RunnableLambda:
second_good_input = RunnableLambda(lambda x: { "film_required_age": x["film_required_age"] } )

third_good_input = RunnablePassthrough()
fourth_good_input = {"film_required_age": RunnablePassthrough()}
# You can also create a chain starting with RunnableParallel

first_good_chain = first_good_input | prompt
second_good_chain = second_good_input | prompt
third_good_chain = third_good_input | prompt
fourth_good_chain = fourth_good_input | prompt

first_good_chain.invoke({
    "film_required_age": 18
})

# ...


ChatPromptValue(messages=[HumanMessage(content='Generate a film title, the age is 18')])

## Order Matters

In [2]:
bad_order_chain = prompt | first_good_input
bad_order_chain.invoke({"film_required_age": 18})

NameError: name 'first_good_input' is not defined

Given that your story generation will require multiple sequential prompts you can use a create `SequentialChain` to chain multiple prompts together. The `SequentialChain` will need to re-use the outputs of the previous prompt and use it as the input for the next prompt.


In [3]:
from langchain_core.prompts.chat import ChatPromptTemplate

In [4]:
character_generation_prompt = ChatPromptTemplate.from_template(
    """I want you to brainstorm 3 - 5 characters for my short story. The genre is {genre}.
    Each character must have a Name and a Biography.
    You must provide a name and biography for each character, this is very important!
    ---
    Example response:
    Name: CharWiz, Biography: A wizard who is a master of magic.
    Name: CharWar, Biography: A warrior who is a master of the sword.
    ---
    Characters: """
)

plot_generation_prompt = ChatPromptTemplate.from_template(
    """Given the following characters and the genre, create an effective plot for a short story:
    Characters:
    {characters} 
    ---
    Genre: {genre}
    ---
    Plot: """
    )

scene_generation_plot_prompt = ChatPromptTemplate.from_template(
    """Act as an effective content creator. 
    Given multiple characters and a plot you are responsible generating the various scenes for each act. 
    
    You must de-compose the plot into multiple effective scenes:

    ---
    Characters:
    {characters}
    ---
    Genre: {genre}
    ---
    Plot: {plot}
    ---
    Example response:
    Scenes:
    Scene 1: Some text here.
    Scene 2: Some text here.
    Scene 3: Some text here.
    ----
    Scenes:
    """
)

---

## ItemGetter and RunnableLambda

In [5]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [6]:
chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
}
chain.invoke({"genre": "fantasy"})

{'genre': 'fantasy'}

In [7]:
# Another way to achieve the same thing:
chain = RunnableLambda(lambda x: {"genre": x["genre"]}) | {"genre": itemgetter("genre")}
chain.invoke({"genre": "fantasy"})
# {'genre': 'fantasy'}

{'genre': 'fantasy'}

In [8]:
chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
    "upper_case_genre": lambda x: x["genre"].upper(),
    "lower_case_genre": RunnableLambda(lambda x: x["genre"].lower()),
}
chain.invoke({"genre": "fantasy"})

{'genre': 'fantasy',
 'upper_case_genre': 'FANTASY',
 'lower_case_genre': 'fantasy'}

---------------------------------------------

## RunnableParallel

In [9]:
from langchain_core.runnables import RunnableParallel

master_chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
    "upper_case_genre": lambda x: x["genre"].upper(),
    "lower_case_genre": RunnableLambda(lambda x: x["genre"].lower()),   
}

master_chain_two = RunnablePassthrough() | RunnableParallel(
        genre=itemgetter("genre"),
        upper_case_genre=lambda x: x["genre"].upper(),
        lower_case_genre=RunnableLambda(lambda x: x["genre"].lower()),
)

story_result = master_chain.invoke({"genre": "Fantasy"})
print(f"master chain: {story_result}")

story_result = master_chain_two.invoke({"genre": "Fantasy"})
print(f"master chain two: {story_result}")

master chain: {'genre': 'Fantasy', 'upper_case_genre': 'FANTASY', 'lower_case_genre': 'fantasy'}
master chain two: {'genre': 'Fantasy', 'upper_case_genre': 'FANTASY', 'lower_case_genre': 'fantasy'}


--------------------------------------------------------------------------------

In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [11]:
# Create the chat model:
model = ChatOpenAI()

# Create the sub-chains:
character_generation_chain = character_generation_prompt | model |  StrOutputParser()
plot_generation_chain = plot_generation_prompt | model | StrOutputParser()                                                               
scene_generation_plot_chain = scene_generation_plot_prompt | model | StrOutputParser()

In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter

master_chain = (
    {"characters": character_generation_chain, "genre": RunnablePassthrough()}
    | RunnableParallel(
        characters=itemgetter("characters"),
        genre=itemgetter("genre"),
        plot=plot_generation_chain,
    )
    | RunnableParallel(
        characters=itemgetter("characters"),
        genre=itemgetter("genre"),
        plot=itemgetter("plot"),
        scenes=scene_generation_plot_chain,
    )
)

story_result = master_chain.invoke({"genre": "Fantasy"})

In [13]:
print(story_result['scenes'])

Scene 1: Elara Moonshadow senses a disturbance in the Moonshadow Forest and sets out to investigate. She encounters Magnus Stormbringer, who is researching his family lineage, and convinces him to join her quest.

Scene 2: Seraphina Frostheart receives a vision of a frozen wasteland consumed by darkness. Determined to prevent this future, she embarks on her own journey. Along the way, she is joined by Garrick Ironfist, who shares her cause.

Scene 3: Elara, Magnus, Seraphina, and Garrick's paths converge, and they realize their combined strength is necessary to confront the looming evil. They form an alliance and set out together to confront the darkness.

Scene 4: Lysandra Shadowbane, initially skeptical of joining the group, is hesitant to trust others. Through a series of encounters and trials, the group earns Lysandra's trust, and she reluctantly agrees to join them.

Scene 5: The group embarks on a perilous journey, facing treacherous landscapes, powerful enemies, and internal con

---


## Sequential Story Scene Generation:


In [14]:
# Extracting the scenes using .split('\n') and removing empty strings:
scenes = [scene for scene in story_result["scenes"].split("\n") if scene]
generated_scenes = []
previous_scene_summary = ""

In [15]:
character_script_prompt = ChatPromptTemplate.from_template(
    template="""Given the following characters: {characters} and the genre: {genre}, create an effective character script for a scene.

    You must follow the following principles:
    - Use the Previous Scene Summary: {previous_scene_summary} to avoid repeating yourself.
    - Use the Plot: {plot} to create an effective scene character script.
    - Currently you are generating the character dialogue script for the following scene: {scene}

    ---
    Here is an example response:
    SCENE 1: ANNA'S APARTMENT

    (ANNA is sorting through old books when there is a knock at the door. She opens it to reveal JOHN.)
    ANNA: Can I help you, sir?
    JOHN: Perhaps, I think it's me who can help you. I heard you're researching time travel.
    (Anna looks intrigued but also cautious.)
    ANNA: That's right, but how do you know?
    JOHN: You could say... I'm a primary source.

    ---
    SCENE NUMBER: {index}

    """,
)

summarize_prompt = ChatPromptTemplate.from_template(
    template="""Given a character script create a summary of the scene. Character script: {character_script}""",
)

------------------------------------------------------------

In [16]:
# Loading a chat model:
model = ChatOpenAI(model='gpt-3.5-turbo-16k')

# Create the LCEL chains:
character_script_generation_chain = (
    {
        "characters": RunnablePassthrough(),
        "genre": RunnablePassthrough(),
        "previous_scene_summary": RunnablePassthrough(),
        "plot": RunnablePassthrough(),
        "scene": RunnablePassthrough(),
        "index": RunnablePassthrough(),
    }
    | character_script_prompt
    | model
    | StrOutputParser()
)

summarize_chain = summarize_prompt | model | StrOutputParser()

# You might want to use tqdm here to track the progress, or use all of the scenes:
for index, scene in enumerate(scenes[0:5]):
    
    # # Create a scene generation:
    scene_result = character_script_generation_chain.invoke(
        {
            "characters": story_result["characters"],
            "genre": "fantasy",
            "previous_scene_summary": previous_scene_summary,
            "index": index,
        }
    )

    # Store the generated scenes:
    generated_scenes.append(
        {"character_script": scene_result, "scene": scenes[index]}
    )

    # If this is the first scene then we don't have a previous scene summary:
    if index == 0:
        previous_scene_summary = scene_result
    else:
        # If this is the second scene or greater then we can use and generate a summary:
        summary_result = summarize_chain.invoke(
            {"character_script": scene_result}
        )
        previous_scene_summary = summary_result

--------------------------------------------------------------------------------

In [17]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import pandas as pd

/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [18]:
df = pd.DataFrame(generated_scenes)

In [19]:
all_character_script_text = "\n".join(df.character_script.tolist())

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=200
)

docs = text_splitter.create_documents([all_character_script_text])

In [21]:
chain = load_summarize_chain(llm=model, chain_type="map_reduce")
summary = chain.invoke(docs)
print(summary)

{'input_documents': [Document(page_content="SCENE 1: THE MYSTICAL MOONSHADOW FOREST\n\n(ELARA is patrolling the forest, her bow and arrows ready. MAGNUS appears, holding a book.)\n\nELARA: (alert) Who goes there?\n\nMAGNUS: (smiling) Fear not, Elara. It is I, Magnus Stormbringer. I come bearing knowledge.\n\nELARA: (relaxing) Magnus, what brings you to the Moonshadow Forest?\n\nMAGNUS: I've been researching ancient spells and legends. I believe there may be a connection between your forest and my magical abilities.\n\nELARA: (curious) Tell me more, Magnus. What have you discovered?\n\nMAGNUS: (excited) According to these ancient texts, the Moonshadow Forest holds the key to unlocking my true potential as a storm mage. The lightning and weather manipulation abilities I possess may be tied to the mystical energies of this place.\n\nELARA: (amazed) That's incredible! Our forest has always been a source of magic and power. Perhaps together, we can uncover the secrets of our lineages.\n\nMA